In [54]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K
from keras.models import Sequential
from keras.layers import LSTM,Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D, Flatten, Reshape
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
from numpy.random import seed
from math import sqrt
import numpy as np # linear algebra
import pandas as pd # data processing
from datetime import datetime
import timeit

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

Symbol = 'AAP'

data = pd.read_csv('dataset/splitted_s&p500/'+Symbol+'.csv')
data.head()

,timestamp,close,high,low,open,volume
0,2017-09-11 09:30:00,NaN,NaN,NaN,NaN,NaN
1,2017-09-11 09:31:00,93.565,94.050,93.560,94.0000,17098.0
2,2017-09-11 09:32:00,94.060,94.060,93.820,93.8900,1600.0
3,2017-09-11 09:33:00,NaN,NaN,NaN,NaN,NaN
4,2017-09-11 09:34:00,94.095,94.498,94.095,94.3569,1519.0


In [55]:
data.size

258876

In [56]:
#fill nan with last val instance
data = data.fillna(method='ffill')

In [41]:
def removeEarly(data):
    for i in range(50):
        if not data.iloc[i,5] == data['volume'][i]:
            return data.iloc[i:, 1:]

In [42]:

# for cases where the first raw is missing
X = pd.DataFrame(removeEarly(data))
# X.columns = ['Date', 'Open','High','Low','Close','Volume']

In [47]:
data.iloc[0:, 1:]

,close,high,low,open,volume
0,NaN,NaN,NaN,NaN,NaN
1,93.5650,94.050,93.5600,94.0000,17098.0
2,94.0600,94.060,93.8200,93.8900,1600.0
3,94.0600,94.060,93.8200,93.8900,1600.0
4,94.0950,94.498,94.0950,94.3569,1519.0
...,...,...,...,...,...
43141,107.8500,107.880,107.8101,107.8200,5916.0
43142,107.8700,107.910,107.8500,107.8650,4368.0
43143,107.8900,107.930,107.8650,107.8650,6332.0
43144,107.8100,107.900,107.8100,107.8600,7287.0


In [1]:
from utils.dataloader import StockDataset

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# Load the data
data = StockDataset(csv_path='dataset/splitted_s&p500/AAP.csv', sequence_length=50, train=True, normalize=False)

In [2]:
data.data

array([[ 93.565,  94.05 ,  93.56 , ...,   9.   ,  31.   ,  93.565],
       [ 94.06 ,  94.06 ,  93.82 , ...,   9.   ,  32.   ,  94.06 ],
       [ 94.06 ,  94.06 ,  93.82 , ...,   9.   ,  33.   ,  94.06 ],
       ...,
       [115.33 , 115.4  , 115.3  , ...,  15.   ,  30.   , 115.33 ],
       [115.37 , 115.38 , 115.32 , ...,  15.   ,  31.   , 115.37 ],
       [115.32 , 115.39 , 115.26 , ...,  15.   ,  32.   , 115.32 ]])

In [5]:
data.data[0].shape

(8,)

In [6]:
import torch

In [7]:
x = torch.utils.data.DataLoader(dataset=data, batch_size=20, shuffle=False)

In [15]:
for i, e in x:
    print(i.shape)
    print(e.shape)
    

torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Size([20, 8])
torch.Size([20, 49, 8])
torch.Si

In [58]:
data

,timestamp,close,high,low,open,volume
1,2017-09-11 09:31:00,93.5650,94.050,93.5600,94.0000,17098.0
2,2017-09-11 09:32:00,94.0600,94.060,93.8200,93.8900,1600.0
3,2017-09-11 09:33:00,94.0600,94.060,93.8200,93.8900,1600.0
4,2017-09-11 09:34:00,94.0950,94.498,94.0950,94.3569,1519.0
5,2017-09-11 09:35:00,94.3400,94.340,94.1900,94.1900,200.0
...,...,...,...,...,...,...
43141,2018-02-16 15:55:00,107.8500,107.880,107.8101,107.8200,5916.0
43142,2018-02-16 15:56:00,107.8700,107.910,107.8500,107.8650,4368.0
43143,2018-02-16 15:57:00,107.8900,107.930,107.8650,107.8650,6332.0
43144,2018-02-16 15:58:00,107.8100,107.900,107.8100,107.8600,7287.0


In [21]:
# Make the closing price of the last 5 days as the target for the 5 days
X['Target'] = X['Close'].shift(-5)

Rolling [window=5,center=False,axis=0,method=single]

In [35]:
len(data) // 5

8629

In [57]:
data.dropna(inplace=True)

In [59]:
data = data.values

IndexError: index 6 is out of bounds for axis 1 with size 6

SyntaxError: EOL while scanning string literal (2089444095.py, line 1)

In [61]:
asd = np.array([])

In [69]:
appa

ValueError: could not broadcast input array from shape (3,) into shape (1,)